In [1]:
as = [3.3 , 4.3] #prices public info
cs = [3.0,4.0] #costs, private info
n= 2
lag = 1
#info set is lagged 1 period back,

function init_Ws(as,cs,n,lag, W0)
    Ws = Dict()
    argumento=fill(as, n*lag + 1 )
    cartesian_as=collect(Iterators.product(argumento...))
    
    argumento= fill(cs, lag + 1 )
    cartesian_cs = collect(Iterators.product(argumento...))
    for c=cartesian_cs
        for a=cartesian_as
                Ws["$(a),$(c)"] = (rand(W0),0)
        end
    end
    return Ws
end

W0 = init_Ws(as,cs,n,lag,[10,11,12])

Dict{Any,Any} with 32 entries:
  "(4.3, 3.3, 3.3),(3.0, 4.0)" => (12, 0)
  "(3.3, 3.3, 4.3),(3.0, 3.0)" => (12, 0)
  "(4.3, 3.3, 3.3),(4.0, 3.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 4.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 3.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 3.3, 4.3),(3.0, 4.0)" => (11, 0)
  "(4.3, 4.3, 4.3),(4.0, 4.0)" => (12, 0)
  "(3.3, 3.3, 4.3),(4.0, 3.0)" => (10, 0)
  "(3.3, 3.3, 4.3),(3.0, 4.0)" => (12, 0)
  "(3.3, 4.3, 3.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 4.3, 4.3),(3.0, 3.0)" => (10, 0)
  "(4.3, 4.3, 3.3),(3.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 3.3),(4.0, 3.0)" => (10, 0)
  "(3.3, 4.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 3.3),(3.0, 3.0)" => (10, 0)
  "(4.3, 4.3, 4.3),(4.0, 3.0)" => (12, 0)
  "(4.3, 4.3, 3.3),(4.0, 4.0)" => (12, 0)
  "(3.3, 4.3, 4.3),(3.0, 4.0)" => (10, 0)
  "(3.3, 4.3, 3.3),(3.0, 3.0)" => (10, 0)
  "(4.3, 4.3, 3.3),(4.0, 3.0)" => (12, 0)
  "

In [2]:
function update_m(as,cs,n,lag,Ws)
    
    argumento=fill(as, n*lag)
    cartesian_as=collect(Iterators.product(argumento...))
    argumento= fill(cs, lag + 1 )
    cartesian_cs = collect(Iterators.product(argumento...))
    
    ms = Dict()
    for other_a=cartesian_as
        for c=cartesian_cs
            for my_a=as
                #println("my_a ", my_a)
                #check if key is there initilize to my_a
                if ! haskey(ms, "$(other_a),$(c)") 
                    ms["$(other_a),$(c)"] = my_a
                end
                
                current_pol = ms["$(other_a),$(c)"]
                current_state = Tuple(append!([current_pol],other_a ))
                other_state = Tuple(append!([my_a],other_a))
                
                #if already there set to my a if greater
                #println(my_a, ", ", current_state,", " ,other_state, ", ", Ws["$(current_state),$(c)"], ", ",  Ws["$(other_state),$(c)"] )
                if Ws["$(current_state),$(c)"][1] <= Ws["$(other_state),$(c)"][1]
                    ms["$(other_a),$(c)"] = my_a
                end
            end
        end
    end
    m(as,omegas) = ms["$(Tuple(as)),$(Tuple(omegas))"]
    return m
end


m0 = update_m(as,cs,n,lag,W0)
print(m0)
m0((4.3, 4.3),(3.,3.))

getfield(Main, Symbol("#m#3")){Dict{Any,Any}}(Dict{Any,Any}("(3.3, 3.3),(3.0, 4.0)"=>4.3,"(4.3, 4.3),(4.0, 4.0)"=>4.3,"(3.3, 4.3),(3.0, 3.0)"=>3.3,"(4.3, 4.3),(3.0, 3.0)"=>3.3,"(3.3, 3.3),(3.0, 3.0)"=>3.3,"(3.3, 3.3),(4.0, 3.0)"=>4.3,"(4.3, 4.3),(3.0, 4.0)"=>4.3,"(4.3, 3.3),(4.0, 3.0)"=>4.3,"(3.3, 4.3),(3.0, 4.0)"=>3.3,"(3.3, 4.3),(4.0, 3.0)"=>4.3,"(3.3, 4.3),(4.0, 4.0)"=>4.3,"(3.3, 3.3),(4.0, 4.0)"=>3.3,"(4.3, 4.3),(4.0, 3.0)"=>4.3,"(4.3, 3.3),(3.0, 4.0)"=>3.3,"(4.3, 3.3),(4.0, 4.0)"=>4.3,"(4.3, 3.3),(3.0, 3.0)"=>4.3))

3.3

In [3]:
#reset costs every 2 periods
function update_private_info(m,current_as,current_cs,cs,n,lag,t)
    next_cs = copy(current_cs)
    
    #update costs
    for i=1:n
        next_ci = collect(next_cs[i])
        next_ci[1] =current_cs[i][1]
        if mod(t,2) == 0
            next_ci[1] = cs[rand(1:end)]
        end

        for s=0:(lag-1)
            next_ci[end-s] = current_cs[i][end-s-1]
        end
        next_cs[i] = Tuple(next_ci)
    end
    return next_cs
end


function update_public_info(m,current_as,current_cs,cs,n,lag,t)
    next_as = copy(current_as)
    for i=1:n
        next_asi = collect(next_as[i])
        for j=0:(n-1)
            next_asi[mod(j+i-1,n)+1] = m(current_as[i],current_cs[i])
        end
        next_as[i] = Tuple(next_asi)
    end
    return next_as
end


function update_info(m,current_as,current_cs,cs,n,lag,t)
    next_cs = update_private_info(m,current_as,current_cs,cs,n,lag,t)
    next_as = update_public_info(m,current_as,current_cs,cs,n,lag,t)
    return next_as, next_cs, t+1
end

update_info(m0, [(4.3, 4.3),(4.3,4.3)],[(4.,4.),(4.,4.)],cs,n,lag,2)

(Tuple{Float64,Float64}[(4.3, 4.3), (4.3, 4.3)], Tuple{Float64,Float64}[(4.0, 4.0), (4.0, 4.0)], 3)

In [4]:
BETA = .9
W0 = init_Ws(as,cs,n,lag,[10,11,12])

function update_ws(m,current_as,current_cs, n, beta, Ws)
    

    profit = zeros(n)
    policies = zeros(n)
    
    for i=1:n
        policies[i] = m( current_as[i], current_cs[i] )
    end
    for i=1:n
        share = 1 - exp(policies[i]) / ( 1+sum(exp.(policies) ))
        state = Tuple( append!( [policies[i]],current_as[i] ) )
        profit[i] = share*(policies[i] - current_cs[i][1]) + beta*Ws["$(state),$(current_cs[i])"][1]
    end
    
    for i=1:n
        state = Tuple( append!( [policies[i]],current_as[i] ) )
        weight = 1/( Ws["$(state),$(current_cs[i])"][2] +1 )
        new_w = weight*profit[i] + (1-weight)*Ws["$(state),$(current_cs[i])"][1]
        new_counter = Ws["$(state),$(current_cs[i])"][2] +1
        Ws["$(state),$(current_cs[i])"]= (new_w,new_counter)
    end
    
    return Ws
end

competition(x,y) = 3.3

update_ws( m0,[(4.3, 4.3),(4.3,4.3)],[(3.,3.),(3.,3.)], n, 0, W0 )

Dict{Any,Any} with 32 entries:
  "(4.3, 3.3, 3.3),(3.0, 4.0)" => (10, 0)
  "(3.3, 3.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(4.0, 3.0)" => (10, 0)
  "(3.3, 4.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 4.3),(4.0, 4.0)" => (12, 0)
  "(4.3, 3.3, 4.3),(3.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 3.3, 4.3),(3.0, 4.0)" => (12, 0)
  "(4.3, 4.3, 4.3),(4.0, 4.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(4.0, 3.0)" => (11, 0)
  "(3.3, 3.3, 4.3),(3.0, 4.0)" => (10, 0)
  "(3.3, 4.3, 3.3),(4.0, 4.0)" => (10, 0)
  "(4.3, 4.3, 4.3),(3.0, 3.0)" => (12, 0)
  "(4.3, 4.3, 3.3),(3.0, 3.0)" => (12, 0)
  "(3.3, 3.3, 3.3),(4.0, 3.0)" => (11, 0)
  "(3.3, 4.3, 4.3),(3.0, 3.0)" => (0.152716, 2)
  "(4.3, 3.3, 3.3),(4.0, 4.0)" => (11, 0)
  "(4.3, 3.3, 3.3),(3.0, 3.0)" => (10, 0)
  "(4.3, 4.3, 4.3),(4.0, 3.0)" => (11, 0)
  "(4.3, 4.3, 3.3),(4.0, 4.0)" => (10, 0)
  "(3.3, 4.3, 4.3),(3.0, 4.0)" => (11, 0)
  "(3.3, 4.3, 3.3),(3.0, 3.0)" => (10, 0)
  "(4.3, 4.3, 3.3),(4.0, 3.0)" => (12, 

In [10]:
iters=10000
as = [3.7,4.7] #prices public info
cs = [3.,4.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[1.8,1.9,2.0])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(3.7, 3.7),(3.7,3.7)]
cs_current = [(3.,3.),(3.,3.)]
t=0
beta = .9

for i=1:iters
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] > 40
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end

(3.7, 3.7, 3.7),(3.0, 3.0)(3.318435469777754, 7419)
(4.7, 3.7, 3.7),(4.0, 3.0)(2.3650433228586873, 2488)
(4.7, 4.7, 4.7),(4.0, 4.0)(2.307966922684105, 7602)
(3.7, 4.7, 4.7),(3.0, 4.0)(3.36218296120743, 2489)
3.7: 9910
4.7: 10090


In [11]:
iters=10000
as = [3.7,4.7] #prices public info
cs = [3.,4.] #costs, private info
n= 2
lag = 1
Ws = init_Ws(as,cs,n,lag,[1.8,1.9,2.0])
ms = update_m(as,cs,n,lag,Ws)
as_current = [(3.7, 3.7),(3.7,3.7)]
cs_current = [(3.,3.),(3.,3.)]
t=0
beta = 0

for i=1:iters
    as_current, cs_current, t = update_info(ms,as_current,cs_current,cs,n,lag,t)
    Ws= update_ws(ms,as_current,cs_current, n, beta, Ws)
    ms= update_m(as,cs,n,lag,Ws)
end

for (k,v) in Ws
    if v[2] > 40
        println(k,v)
    end
end

for a=as
    tally = 0
    for (k,v) in Ws
        action = parse(Float64,k[2:4])
        if action ==a
            tally = tally + v[2]
        end
    end
    println("$(a): $(tally)")
end

(4.7, 4.7, 4.7),(3.0, 4.0)(0.853847993484707, 2485)
(4.7, 4.7, 4.7),(4.0, 4.0)(0.35154269582655223, 7658)
(4.7, 4.7, 4.7),(3.0, 3.0)(0.8538479934847039, 7360)
(4.7, 4.7, 4.7),(4.0, 3.0)(0.3515201553641898, 2487)
3.7: 7
4.7: 19993
